#  Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
from collections import Counter
import tensorflow as tf
from tensorflow.keras import Model, models, layers, optimizers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

seed=1
tf.random.set_seed(seed)

# Load Data

In [3]:
train = pd.read_csv('2_only_normal_threshold02.csv')
test = pd.read_csv('2_test.csv')

In [4]:
train.drop(['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1)
test.drop(['Unnamed: 0', 'Timestamp'], inplace=True, axis=1)

In [5]:
shift_by = -5
n_features = train.shape[1]

# Transform to Series Data 

In [6]:
shift_by = -5
n_features = train.shape[1]

In [7]:
# LSTM 모델은 (samples, timesteps, feature)에 해당하는 3d 차원의 shape을 가지므로, 데이터를 시퀀스 형태로 변환

def temporalize(X, timesteps):
    output_X = []
    for i in range(len(X) - timesteps - 1):
        t = []
        for j in range(1, timesteps + 1):
            # Gather the past records upto the lookback period
            t.append(X.loc[[(i + j + 1)], :])
        output_X.append(t)
    return np.squeeze(np.array(output_X))

In [8]:
timesteps = abs(shift_by)

# Temporalize
x = temporalize(train, timesteps)
print(x.shape)

(180006, 5, 33)


# Split Train / Valid / Test

In [9]:
# Split into train, valid, and test 
x_train = x
x_test = temporalize(test, timesteps)
x_train, x_valid = train_test_split(x_train, test_size=0.2)

print(len(x_train))  
print(len(x_valid)) 
print(len(x_test))

144004
36002
105114


# Standardize

In [10]:
def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)

def scale(X, scaler):
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])
        
    return X

In [11]:
scaler = MinMaxScaler().fit(flatten(x_train))

x_train_scaled = scale(x_train, scaler)
x_valid_scaled = scale(x_valid, scaler)
x_test_scaled = scale(x_test, scaler)

# Training LSTM Autoencoder

In [12]:
epochs = 10
batch = 128
lr = 0.001

In [13]:
# earlystopping
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

lstm_ae = models.Sequential()
# Encoder
lstm_ae.add(layers.LSTM(64, activation='relu', input_shape=(timesteps, n_features), return_sequences=True))
lstm_ae.add(layers.LSTM(32, activation='relu', return_sequences=False))
lstm_ae.add(layers.RepeatVector(timesteps))
# Decoder
lstm_ae.add(layers.LSTM(32, activation='relu', return_sequences=True))
lstm_ae.add(layers.LSTM(64, activation='relu', return_sequences=True))
lstm_ae.add(layers.TimeDistributed(layers.Dense(n_features)))

lstm_ae.summary()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-05-10 11:58:19.311769: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-10 11:58:19.313810: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 64)             25088     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 repeat_vector (RepeatVector  (None, 5, 32)            0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 5, 32)             8320      
                                                                 
 lstm_3 (LSTM)               (None, 5, 64)             24832     
                                                                 
 time_distributed (TimeDistr  (None, 5, 33)            2145      
 ibuted)                                                

In [14]:
# compile
lstm_ae.compile(loss="mse", optimizer=optimizers.Adam(lr))

# fit
history = lstm_ae.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=epochs,
    batch_size=batch,
    validation_data=(x_valid_scaled, x_valid_scaled),
    callbacks=[callback],
)

Epoch 1/10


2022-05-10 11:58:19.786361: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-10 11:58:20.628478: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1126/1126 [==============================] - ETA: 0s - loss: 0.0013

2022-05-10 12:01:34.640662: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1126/1126 [==============================] - 201s 177ms/step - loss: 0.0013 - val_loss: 2.4705e-04
Epoch 2/10
1126/1126 [==============================] - 191s 169ms/step - loss: 1.8603e-04 - val_loss: 1.6873e-04
Epoch 3/10
1126/1126 [==============================] - 217s 193ms/step - loss: 1.4092e-04 - val_loss: 1.5832e-04
Epoch 4/10
1126/1126 [==============================] - 210s 187ms/step - loss: 1.2668e-04 - val_loss: 1.3261e-04
Epoch 5/10
1126/1126 [==============================] - 212s 188ms/step - loss: 1.1770e-04 - val_loss: 1.1449e-04
Epoch 6/10
1126/1126 [==============================] - 213s 189ms/step - loss: 1.1148e-04 - val_loss: 1.5677e-04
Epoch 7/10
1126/1126 [==============================] - 214s 190ms/step - loss: 1.1165e-04 - val_loss: 1.3018e-04
Epoch 8/10
1126/1126 [==============================] - 215s 191ms/step - loss: 1.0760e-04 - val_loss: 1.2050e-04
Epoch 9/10
1126/1126 [==============================] - 215s 191ms/step - loss: 1.0491e-04 - val_loss: 

# threshold

In [61]:
threshold = 0.1

In [16]:
test.shape

(105120, 33)

In [17]:
test_predictions = lstm_ae.predict(x_test_scaled)

2022-05-10 12:33:22.180698: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [62]:
mse = np.sqrt(np.mean(np.power(test_predictions.reshape(test_predictions.shape[0], test_predictions.shape[1] * test_predictions.shape[2]) - x_test_scaled.reshape(test_predictions.shape[0], test_predictions.shape[1] * test_predictions.shape[2]), 2)))
print(mse)

0.016519000903604128


In [63]:
predict = []
for i in range(len(x_test_scaled)):
    if np.sqrt(np.mean(np.power(test_predictions.reshape(test_predictions.shape[0], test_predictions.shape[1] * test_predictions.shape[2])[i] - x_test_scaled.reshape(test_predictions.shape[0], test_predictions.shape[1] * test_predictions.shape[2])[i], 2))) > threshold:
        predict.append(1)
    else:
        predict.append(0)

In [64]:
predict = pd.DataFrame(predict, columns=['Prediction'])
predict.value_counts()

Prediction
0             104889
1                225
dtype: int64

In [65]:
predict[predict['Prediction']==1]

,Prediction
2226,1
2227,1
2228,1
2229,1
2230,1
...,...
96186,1
96187,1
96188,1
96189,1


In [66]:
predict.to_csv('2_predict5.csv', mode='w')

In [67]:
ori = pd.read_csv('2_predict_ori.csv')

In [68]:
ori['Prediction'].value_counts()

0    104825
1       295
Name: Prediction, dtype: int64

In [69]:
conf = pd.read_csv('2_predict5.csv')

In [70]:
diff = []
for i in range(len(ori)) :
    if ori["Prediction"][i] != conf["Prediction"][i] :
        diff.append(i)
        
print(len(diff))

80
